In [6]:
import tensorflow as tf
import joblib
import numpy as np
import matplotlib.pyplot as plt
import seaborn

file = (
    '/home/vitchyr/git/rllab-rail/railrl/data/local'
    '/6-16-launch-benchmark-1d'
    '/6-16-launch-benchmark-1d_2017_06_16_17_36_44_0002--s-0'
    '/params.pkl'

)

tf.reset_default_graph()
sess = tf.InteractiveSession()

data = joblib.load(file)
policy = data['policy']
qf = data['qf']
env = data['env']
print("Policy loaded")


KeyError: 'qf'